# Convert MAGS (Vega or AB mag system) to Flux
You can use the customized package what_the_flux (written by Chris Frohmaier, with a small contribution from Maria Vincenzi).

Here you can find a quick example.

Remember that:
1) 90% of the times Core Collapse SNe are presented in the standard filter system (not in natural filter system). This means that photometry is presented as it would have been observed by the "standard" Bessell/SDSS filter set. The "standard" Bessell/SDSS filter sets are stored in inputs/Filters/GeneralFilters. The main exception is CSP data, usually presented in natural system. Therfore, for CSP objects we use inputs/Filters/Site3_CSP.

2) Magnitudes in Bessell filters (UBVRI) are usually calculated in the Vega system and calibrated to Landolt standards

3) Magnitudes in SDSS filters (ugriz) are usually calculated in the AB system and calibrated to Smith 2002 standards

4) Magnitudes in Space Telescope SWIFT filters (W2,M2,W1,U,B,V) are usually calculated in the Vega system. See Peter Brown's paper for calibration of the instrument.


In [7]:
COCO_PATH='/Volumes/TOSHIBAEXT/CoCoTmpl/'

In [8]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, COCO_PATH+'what_the_flux/')
import what_the_flux as wtf

In [17]:
input_LC_path = COCO_PATH+"inputs/Photometry/0_LCs_mags_raw/iPTF13bvn_mag.dat"
output_LC_path = COCO_PATH+"inputs/Photometry/1_LCs_flux_raw/iPTF13bvn.dat"
filters_data_path = COCO_PATH+"inputs/Filters/"

In [39]:
fout = open(output_LC_path, 'w')
fout.write('# From Mags to Flux, not MW/Host dust corrected\n')
fout.close()

phot_mags= pd.read_csv(input_LC_path)
filter_set = 'GeneralFilters'
                
fluxes = []
fluxes_err = []
                
for index, row in phot_mags.iterrows():
    w,t = wtf.loadFilter(filters_data_path+'%s/%s.dat'%(filter_set, row['band']))
    band = wtf.Band_Vega(w,t)
    flux_withMW = wtf.mag2flux(band, row['Mag']).value
    flux_err_withMW = wtf.ERRmag2ERRflux(band, row['Mag'], row['Mag_err']).value
    fluxes.append(flux_withMW)
    fluxes_err.append(flux_err_withMW)
                
phot_mags['Flux'] = fluxes
phot_mags['Flux_err'] = fluxes_err
phot_mags.to_csv(output_LC_path, index=False)